In [1]:
import math

In [2]:
def inverse(x, p):
    for i in range(1, p):
        if ((x * i) % p) == 1:
            return i
    raise Exception('No inverse exists.')

In [3]:
def negative(P, p):
    if not P is None:
        return (P[0], (-1 * P[1]) % p)
    else:
        return P

In [4]:
def double(P, c_4, p):

    numerator = (3 * (P[0] ** 2) + c_4) % p
    denominator = (2 * P[1]) % p

    l = (numerator * inverse(denominator, p)) % p

    x = (l ** 2 - 2 * P[0]) % p
    y = (l * (P[0] - x) - P[1]) % p

    return (x, y)

In [5]:
# P = (x_1, y_1) -> P[0] = x_1, P[1] = y_1
# Q = (x_2, y_2) -> Q[0] = x_2, Q[1] = y_2

def add(P, Q, p):

    numerator = (P[1] - Q[1]) % p
    denominator = (P[0] - Q[0]) % p

    l = (numerator * inverse(denominator, p)) % p

    x = (l ** 2 - P[0] - Q[0]) % p
    y = (l * (P[0] - x) - P[1]) % p

    return (x, y)

In [6]:
# P = (x_1, y_1) -> P[0] = x_1, P[1] = y_1
# Q = (x_2, y_2) -> Q[0] = x_2, Q[1] = y_2

def weierstrass_addition(P, Q, c_4, p):

    if P is None:
        return Q
    if Q is None:
        return P
    if P[0] == Q[0] and P[1] == ((-1 * Q[1]) % p):
        return None
    elif P[0] == Q[0] and P[1] == Q[1]:
        return double(P, c_4, p)
    else:
        return add(P, Q, p)

In [7]:
def order(P, c_4, p):

    sum = P
    order = 1

    while not sum is None:
        sum = weierstrass_addition(sum, P, c_4, p)
        order += 1

    return order

In [8]:
def generate_steps(P, P_A, c_4, p):

    N = order(P, c_4, p)
    m = math.floor(math.sqrt(N))

    left_side = [None]
    for i in range(m - 1):
        left_side.append(weierstrass_addition(left_side[i], P, c_4, p))

    right_side = [P_A]
    mP = weierstrass_addition(left_side[-1], P, c_4, p)
    n_mP = negative(mP, p)

    for i in range(math.floor(N / m) + 1):
        right_side.append(weierstrass_addition(right_side[i], n_mP, c_4, p))

        if right_side[-1] is None:
            a_0 = 0
            a_1 = len(right_side) - 1
            a = a_0 + m * a_1

            return left_side, right_side, a_0, a_1, a

        for j in range(1, len(left_side)):
            a = right_side[-1]
            b = left_side[j]

            if a[0] == b[0] and a[1] == b[1]:
                a_0 = j
                a_1 = len(right_side) - 1
                a = a_0 + m * a_1

                return left_side, right_side, a_0, a_1, a

In [9]:
def print_results(left_side, right_side, a_0, a_1, a, P, P_A, c_4, p):

    for i in range(len(left_side)):
        print(str(i) + 'P = ' + str(left_side[i]))
    print('a_0 = ' + str(a_0))
    print()

    for i in range(len(right_side)):
        print('P_A - ' + str(i) + 'mP = ' + str(right_side[i]))
    print('a_1 = ' + str(a_1))
    print()

    print('a = ' + str(a))
    Q = None
    for i in range(a):
        Q = weierstrass_addition(Q, P, c_4, p)
        print(str(i + 1) + 'P = ' + str(Q))

    print()
    print('P_A = ' + str(P_A))

In [10]:
# y^2 = x^3 + x + 3 over F_43
p = 43
c_4 = 1

In [11]:
P = (19, 42)
P_A = (28, 15)

In [12]:
left_side, right_side, a_0, a_1, a = generate_steps(P, P_A, c_4, p)

In [13]:
print_results(left_side, right_side, a_0, a_1, a, P, P_A, c_4, p)

0P = None
1P = (19, 42)
2P = (36, 13)
3P = (29, 13)
4P = (9, 15)
5P = (21, 30)
a_0 = 3

P_A - 0mP = (28, 15)
P_A - 1mP = (29, 13)
a_1 = 1

a = 9
1P = (19, 42)
2P = (36, 13)
3P = (29, 13)
4P = (9, 15)
5P = (21, 30)
6P = (39, 35)
7P = (2, 23)
8P = (17, 26)
9P = (28, 15)

P_A = (28, 15)


Extra

In [14]:
def DLP_BSGS(P, P_A, c_4, p):
    left_side, right_side, a_0, a_1, a = generate_steps(P, P_A, c_4, p)
    return a